In [1]:
# load csv  as panda df and sort by column
import pandas as pd
import sys
import math
from helper import *

def load_csv(file):
    df = pd.read_csv(file)
    df = df.sort_values(by=["acc_version", "hardware", "model", "thread"])
    df = df.sort_values(by=["thread", "model", "acc_version", "hardware"])
    df.fillna(0, inplace=True)
    valid_loc = df.columns.get_loc("valid")
    for i in range(valid_loc, len(df.columns)):
        df.iloc[:, i] = df.iloc[:, i].astype(int)
    df = df.drop(df.columns[[0]], axis=1)
    df = df.drop(columns=["del", "del_version"])
    return df


def combine(df, df2):
    df2 = df2[df.columns]
    df = pd.concat([df, df2], ignore_index=True)
    return df


def merge_rows(df):
    # combine acc_version, hardware into one col
    df["acc_version"] = df["acc_version"].astype(str)
    df["hardware"] = df["hardware"].astype(str)
    df["acc_version_hardware"] = df["acc_version"] + "_" + df["hardware"]
    # get unique acc_version_hardware
    acc_version_hardware = df["acc_version_hardware"].unique()
    # get unique model
    model = df["model"].unique()

    # create new df with model thread and column for each acc_version_hardware
    df2 = pd.DataFrame(columns=["model", "thread"] + list(acc_version_hardware))

    # for each model and thread, find the total_latency for each acc_version_hardware
    for m in model:
        for t in [1, 2]:
            # get the rows with the same model and thread
            df3 = df.loc[(df["model"] == m) & (df["thread"] == t)]
            # create a row for the model and thread
            df2 = pd.concat(
                [df2, pd.DataFrame([[m, t]], columns=["model", "thread"])],
                ignore_index=True,
            )
            # get the total_latency for each acc_version_hardware
            for a in acc_version_hardware:
                # get the rows with the same acc_version_hardware
                df4 = df3.loc[df3["acc_version_hardware"] == a]
                # get the sum of the total_latency
                total_latency = df4["total_latency"].sum()
                # add the total_latency to the df2 acc_version_hardware column
                df2.loc[(df2["model"] == m) & (df2["thread"] == t), a] = total_latency

    # sort by CPU
    df2 = df2.sort_values(by=["1_0_CPU", "model", "thread"])
    df2 = custom_tconv_cols(df2)
    # #save to csv
    df2.to_csv("2.csv", index=False)


# load csv
# df = load_csv("../results/latest.csv")
# df = load_csv("../results/mm2imv2.csv")
# latest = load_csv("../results/latest.csv")
tconv_2 = load_csv("../saved_results/tconv_2.csv")
mm2imv2_1 = load_csv("../saved_results/mm2imv2_1_post_threading.csv")

df3 = combine(mm2imv2_1, tconv_2)
df3.to_csv("1.csv", index=False)

# visualize
pd.set_option("display.precision", 0)

# create new df which combines the same model, with new col for each thread's total_latency
# df = combine(df,"../results/latest.csv")

# merge rows
merge_rows(df3)

In [ ]:
def get_df_from_csv(csv_path):
    try:
        f = open(csv_path, "r")
    except:
        print(csv_path + " does not exist")
        exit(1)
    save = 0
    savedString = ""
    while True:
        line = f.readline()
        if save == 2:
            savedString += line
        if (
            line
            == "============================== Summary by node type ==============================\n"
        ):
            save += 1
        if not line:
            break
    node_table = savedString.split("\n\n")[0].split("\n")
    node_table = [x.replace(" ", "").split(",") for x in node_table]
    df = pd.DataFrame.from_records(node_table)
    df.columns = df.iloc[0]
    df = df[1:]
    return df

df = get_df_from_csv("../tmp/MM2IM_1_0_mm2im_fpga_delegate_1_tconv_1_1_16_7_7_7_32_1_1_layer.csv")
print(df)


0           nodetype count avg_ms       avg%      cdf% memKB timescalled
1  MM2IMFPGADelegate     1  0.682   89.8551%  89.8551%     0           1
2      STRIDED_SLICE     1  0.047   6.19236%  96.0474%     0           1
3         DEQUANTIZE     1  0.013   1.71278%  97.7602%     0           1
4               PACK     1  0.012   1.58103%  99.3412%     0           1
5              SHAPE     1  0.005  0.658762%      100%     0           1
